# Deeper Diver: DSP & PYNQ

![](assets/transitions/lab_trans_dsp_pynq.svg)

>In the previous notebook, we used SciPy to analyse an audio recording of 2 birds and used filtering to isolate one of them. In this notebook the same techniques will be used but, this time, we'll be moving the software FFT and FIR functions over to FPGA hardware and controlling them using PYNQ as if they were software!

## Reusing Code

As this is a whole new notebook, we will need to load in our audio file again and define our plotting functions, so we can just reuse code from the previous notebook rather than writing it all again (we'll only be using frequency domain plots here to keep things simple).

In [ ]:
from scipy.io import wavfile
import numpy as np

fs, aud_in = wavfile.read("assets/pdd/birds.wav")

In [ ]:
import plotly_express as px
import pandas as pd
import numpy as np

# Derive a custom plotting template from `plotly`
import plotly.io as pio
new_template = pio.templates['plotly']
new_template.update(dict(layout = dict(
        width         = 800,
        autosize      = False,
        legend        = dict(x=1.1),
)))

# Register new template as the default
pio.templates['dsp_plot'] = new_template
pio.templates.default = 'dsp_plot'

def to_freq_dataframe(samples, fs):
    """Create a pandas dataframe from an ndarray frequency domain samples"""
    sample_freqs = np.linspace(0, fs, len(samples))
    return pd.DataFrame(dict(
        amplitude = samples[0:int(len(samples)/2)],  
        freq      = sample_freqs[0:int(len(samples)/2)]
    ))

## Moving to Hardware

The hardware design consists of a LogiCore FIR Compiler (with 99 reprogrammable weights), and a LogiCore FFT (with a fixed length of $2^{14}$). These are then connected, via AXI4-Stream, to the standard AXI DMAs that allow shared access to PS memory. The IPI diagram below shows how the filter and FFT IPs are both within their own hierarchy, this is for two reasons: One is to keep the top diagram simple and easy to follow, the other it makes referencing the IPs in PYNQ a little simpler.

<img src="assets/pdd/dsp_pynq_top.png" width="1200"/>

Of course, to get a better idea of how the hardware is set up, it's best to show you what is inside one of these hierarchies - let's look at the filter. You'll notice straight away that there's not a whole lot in here, just the FIR and a few DMAs for transferring our audio and configuration data. With PYNQ, this is all you need to be able to send data to and from your IP using AXI4-Stream!

<img src="assets/pdd/dsp_pynq_filt.png" width="900"/>


If you want to view these diagrams in a higher resolution, open the file browser and double click on the images in the *assets/pdd* directory. This will open them in a new tab.

To start working with our design, we need to first download our bitstream onto the FPGA. PYNQ makes things simple by giving us the Overlay class...

In [ ]:
from pynq import Overlay

ol = Overlay('assets/pdd/pynq_dsp.bit')

As easy as that! PYNQ also makes use of the accompanying Hardware Handoff (HWH) file to create a dictionary of the IP in the design. This helps with identifying any IP for which drivers can be automatically assigned, such as the DMAs. You can view the keys for IP dictionary by running the command in the cell below.

In [ ]:
list(ol.ip_dict.keys())

Another great feature of PYNQ is having quick access to the entire register map of any IP in your design. As an example, let's have a look at our "fft_data" DMA. You can refer to [PG021](https://www.xilinx.com/support/documentation/ip_documentation/axi_dma/v7_1/pg021_axi_dma.pdf#page=12) to check these results.

In [ ]:
ol.fft.fft_data.register_map

Now that we have a better idea of how the hardware looks, let's start off with the FFT and check our signal in the frequency domain...

## Hardware FFT

First off we need to create the DMA objects for the FFT IP. There are two associated DMAs here, one for data, and the other for configuration. 

In [ ]:
fft_data = ol.fft.fft_data
fft_config = ol.fft.fft_config

We also need to create a Xlnk object, which is used to allocate PS memory for our IP on the PL.

In [ ]:
from pynq import Xlnk

xlnk = Xlnk()

The IP will be set up for a forward FFT with a selected scaling schedule. We do this by sending a single, 16-bit packet to the FFT AXI4-Stream config port. This involves a few steps:
  - First we create our config packet (in this case it's easier to show in binary) 
  - Next we create a contiguous memory buffer using the Xlnk object
  - Then we fill our buffer with the config packet
  - Finally we transfer our packet to the DMA
    
To learn more about the FFT configuration, you can refer to [PG109](https://www.xilinx.com/support/documentation/ip_documentation/xfft/v9_1/pg109-xfft.pdf#page=16). And you can learn more about the DMA class in the [PYNQ documentation](https://pynq.readthedocs.io/en/v2.1/pynq_libraries/dma.html).



In [ ]:
def get_config_value(forwards, scaling_sched):
    val = 0
    for scaling in scaling_sched:     # [14:1] = scaling schedule
        val = (val << 2) + scaling
    return (val << 1) + int(forwards) # [0] = direction

config_value = get_config_value(True, [1, 1, 2, 2, 2, 2, 2])

fft_buffer_config = xlnk.cma_array(shape=(1,),dtype=np.int16)

fft_buffer_config[0] = config_value

fft_config.sendchannel.transfer(fft_buffer_config)
fft_config.sendchannel.wait()

### Using the FFT

The LogiCore FFT IP data port expects a complex number with 16-bit components (32-bits in total) with the real part in the lower 2-bytes. It returns an equivalent complex 32-bit output as well. 

As our input values are real only, we can just convert the signal to 32-bit values, ensuring the upper 2-bytes are 0.

In [ ]:
# Imag[31:16] Real[15:0] --- imag all zeros
aud_hw = np.asarray(aud_in,np.int32)

Now all we need to do is set up our FFT data buffers and transfer the data. Our FFT is set up in *Non Real Time* throttle scheme, so for every frame of data we transmit, we need to read a frame of data out. 

This would be simple if our signal was an exact multiple of a frame length (in our case 16384 samples), but unfortunately that rarely happens in the real world. To counteract this mismatch in length we need to append zeros to our signal up to the next frame length.

We can do this all within one function as shown in the next cell. You will recognise some of the syntax from the DMA transfer when we sent the configuration packet. The only difference here is that we also expect a packet *back* from the DMA this time as well, so we need to set up an output buffer.

In [ ]:
def fft_hw(signal, NFFT):
    # calculate how many NFFT frames are needed to iterate through entire signal
    max_iters = np.int16(np.ceil(len(signal)/NFFT))
    # calculate amount of zeros to add to make up to NFFT multiple
    zeros = np.int16(np.ceil(len(signal)/NFFT))*NFFT - len(signal)
    # increase length to multiple of NFFT
    signal = np.int32(np.append(signal, np.zeros(zeros)))
    
    fft_in_buffer = xlnk.cma_array(shape=(NFFT,),dtype=np.int32)
    fft_out_buffer = xlnk.cma_array(shape=(NFFT*2,),dtype=np.int16)
    
    fft_out = np.zeros(len(fft_out_buffer))
    
    for i in range(0,max_iters):
        np.copyto(fft_in_buffer,signal[NFFT*i:(NFFT*(i+1))])
        
        fft_data.sendchannel.transfer(fft_in_buffer)
        fft_data.recvchannel.transfer(fft_out_buffer)
        
        fft_data.sendchannel.wait()
        fft_data.recvchannel.wait()
        
        fft_out = fft_out + np.array(fft_out_buffer)
        
    fft_out_buffer.close()
    fft_in_buffer.close()
        
    return fft_out

It's important to note here that the function we just created now works in the same way as the SciPy equivalent that we used in the last notebook (i.e. it takes the same data and the same arguments) - but this time the FFT is in hardware. This really shows the power of PYNQ here: that you can so easily switch between hardware and software and never feel like you've moved between either!

With that said, let's apply the hardware FFT to our audio file...

In [ ]:
NFFT = 2**14
# only perform FFT over small subset of data
aud_fft = fft_hw(aud_hw[np.int16(fs*0.3):np.int16(fs*0.718)],NFFT)

You may have noticed in the function definition that the output buffer is comprised of 16-bit integers while the input buffer has 32-bit integers. You may also have noticed that the length of the output array is double that of the input. Why are we doing this? Well, this is an intentional exploitation of a NumPy feature, where our 32-bit value will be reshaped into two 16-bit values. We use this to make it easier for ourselves to combine the complex output values together, seen in the cell below...

In [ ]:
# make complex number x[n] + j*x[n+1]
aud_fft_c = np.int16(aud_fft[0::2])+1j*np.int16(aud_fft[1::2])

We then take the magnitude of our complex values and plot the results.

In [ ]:
aud_fft_abs = np.abs(aud_fft_c)

# Plot FFT
px.line(
    to_freq_dataframe(aud_fft_abs, fs),
    x='freq', y='amplitude',
    labels = dict(amplitude='Amplitude', freq='Freq (Hz)')
)

Perfect! Now let's move onto filtering the signal with our hardware FIR...

## Hardware FIR Filter

The LogiCore FIR Compiler gives the user the ability to load and reload filter coefficients, on-the-fly, over AXI4-Stream. In this section we use this functionality to filter our audio data in hardware as well.

### Configuring the FIR

Similar to our FFT, we first have to set up the DMAs associated with the FIRs. There are 3 DMAs here, one for data and two for configuration. 

In [ ]:
dma_data = ol.filter.fir_data
dma_config = ol.filter.fir_config
dma_reload = ol.filter.fir_reload

One of the greatest benefits of using PYNQ is that it encourages us to mix our software and hardware in ways rarely implemented before. Remember the coefficients we designed with SciPy's `firwin` function in the previous notebook? We can use those to program the FIR in hardware!

The FIR Compiler is set up to accept 16-bit integers, so we will first need to convert them from their original type of float.

In [ ]:
hpf_coeffs = np.load('assets/pdd/hpf_coeffs.npy')

hpf_coeffs_hw = np.int16(hpf_coeffs/np.max(abs(hpf_coeffs)) * 2**15 - 1)

and because our filter is symmetrical we need only to send half the weights and the FIR compiler will infer the rest...

In [ ]:
hpf_coeffs_hw = hpf_coeffs_hw[0:int(len(hpf_coeffs_hw)/2)+1] # 1/2 + 1 weights

Now we can load the new coefficients to the FIR IP over AXI4-Stream using the same DMA transfer routine we used when configuring the FFT, albeit with an extra step. This *reload/config* transfer is explained in more detail in [PG149](https://www.xilinx.com/support/documentation/ip_documentation/fir_compiler/v7_2/pg149-fir-compiler.pdf#page=16).

In [ ]:
# Create DMA buffer for coefs
fir_buffer_reload = xlnk.cma_array(shape=(len(hpf_coeffs_hw),),dtype=np.int16)

# Copy coefs to buffer
for i in range(len(hpf_coeffs_hw)):
    fir_buffer_reload[i] = hpf_coeffs_hw[i]

# Transfer coefficients to FIR
dma_reload.sendchannel.transfer(fir_buffer_reload)
dma_reload.sendchannel.wait()

# Send an empty 8-bit packet to FIR config port to complete reload
fir_buffer_config = xlnk.cma_array(shape=(1,),dtype=np.int8)
fir_buffer_config[0] = 0
dma_config.sendchannel.transfer(fir_buffer_config)
dma_config.sendchannel.wait()

# Close the buffers
fir_buffer_reload.close()
fir_buffer_config.close()

### Using the FIR

Now we can try filtering the signal using our weights from SciPy. First we need to convert our `aud_hw` signal back to a 16-bit integer then, similarly to how we transferred data to and from the FFT, can do the same for our filter. 

>You'll notice that, as with the FFT, the output buffer is again a different type from the input. In this case the 32-bit output is to take into account bit growth during the filtering process. 

In [ ]:
aud_hw = np.int16(aud_hw)

# Create DMA buffer
fir_in_buffer = xlnk.cma_array(shape=(len(aud_hw),),dtype=np.int16)
fir_out_buffer = xlnk.cma_array(shape=(len(aud_hw),),dtype=np.int32)

# Copy aud_hw to buffer
for i in range(len(aud_hw)):
    fir_in_buffer[i] = aud_hw[i]

# Transfer
dma_data.sendchannel.transfer(fir_in_buffer)
dma_data.recvchannel.transfer(fir_out_buffer)
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()

Then all we need to do is send the result from our FIR to our hardware FFT function, and then plot it!

In [ ]:
# increase the dynamic range and send to FFT
aud_hpf = np.int16(fir_out_buffer/np.max(abs(fir_out_buffer)) * 2**15 - 1)
# only perform FFT over small subset of data
hpf_fft = fft_hw(aud_hpf[np.int16(fs*0.3):np.int16(fs*0.718)],NFFT)

# make complex number x[n] + j*x[n+1]
hpf_fft_c = np.int16(hpf_fft[0::2])+1j*np.int16(hpf_fft[1::2])

# Plot FFT
px.line(
    to_freq_dataframe(np.abs(hpf_fft_c), fs),
    x='freq', y='amplitude',
    labels = dict(amplitude='Amplitude', freq='Freq (Hz)')
)

We can see from the new FFT plot that the lower frequencies have been removed by our hardware filter! We can also check the output by converting the signal back to audio.

In [ ]:
from IPython.display import Audio

scaled = np.int16(fir_out_buffer/np.max(abs(fir_out_buffer)) * 2**15 - 1)
wavfile.write('assets/pdd/hpf_hw.wav', fs, scaled)
Audio('assets/pdd/hpf_hw.wav')

Perfect! If you have time, try out the bandpass filter on your own and plot the results.

## Summary 

Let's recap what we've covered in this last *Deeper Dive*:

  * Reusing code between notebooks (including filter coefficients)
  * Getting to know unique PYNQ features
      + Viewing the IP dictionary and register maps
      + Transferring data between PS and PL using the DMA class
      + IP reconfiguration
  * Creating Python functions for FPGA hardware

> Now we've come to the end of the workshop, we can review what has been covered over the five notebooks. We've demonstrated some of the key features of the RFSoC, including the RF data converters and SD-FECs, and shown how PYNQ can be used to interact with them in a simple and intuitive way. We have also shown off some of the more interesting features of Python and what a typical workflow looks like when using it for DSP development. At the heart of it all is JupyterLab, which gives us the perfect platform on which to show features with text, images, and visualisation - making PYNQ a great tool for demonstrating Xilinx hardware.
      
[⬅️ Previous](04_dsp_and_python.ipynb) 👩‍💻  Next ➡️